In [1]:
import pandas as pd
import os
import gc
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import GPUtil

import torch
import gpytorch

from scipy.stats import pearsonr
from scipy.special import binom as binom
from sklearn.metrics import r2_score as r2
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split

In [2]:
import sys
sys.path.insert(1, '../')
import EpiK.models as models

In [3]:
output_device = 0
n_devices = torch.cuda.device_count()
models.set_params(output_device, n_devices)
print("number of GPUs = {}; output device = {}".
      format(n_devices, torch.cuda.current_device()))

number of GPUs = 8; output device = 0


In [4]:
from EpiK.functions import get_data, get_envs, set_data_path
set_data_path("../matsui_data/")

In [5]:
# training sizes
props = [.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95]

In [6]:
# check_point sizes
partitions = [2, 2, 2, 2, 4, 4, 4, 4, 16, 32, 64, 120]
pd.DataFrame({"props":props, "partitions":partitions}).to_csv("partition_sizes.csv", index=None)

In [7]:
partition_sizes = pd.read_csv("partition_sizes.csv")

### Data

In [8]:
env_list = get_envs()
env = env_list[5]

In [9]:
geno_t, pheno = get_data(env)

/blue/juannanzhou/EpiK/notebooks/../EpiK/functions.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  geno_t = torch.tensor(geno_t, dtype=torch.float)


In [12]:
inds_sub = np.where(np.array(pheno.pheno < -0.6) == False)[0]

In [13]:
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 |  0% | 22% |
|  1 |  0% |  0% |
|  2 |  0% |  0% |
|  3 |  0% |  0% |
|  4 |  0% |  0% |
|  5 |  0% |  0% |
|  6 |  0% |  0% |
|  7 |  0% |  0% |


In [43]:
np.round(props[1]*len(inds_sub)).astype('int')

9397

In [42]:
for i in range(0, len(props)):

    np.random.seed(100)
    train_size = np.round(props[i]*len(inds_sub)).astype('int')
    sub = np.random.choice(inds_sub, train_size)
    sub_t = np.random.choice(list(set(inds_sub).difference(sub)), 5000)
    pd.DataFrame(sub).to_csv(data_path + "train_list_" + str(i) + ".csv", header=None, index=None)
    pd.DataFrame(sub_t).to_csv(data_path + "test_list_" + str(i) + ".csv", header=None, index=None)


In [22]:
data_path = "../BGLR/Data/"

In [27]:
pheno.shape

(190050, 7)

In [30]:
pheno.to_csv(data_path + "pheno_NaCl2.csv", header=True, index=None)

In [41]:
np.unique(geno_t[100].cpu().numpy(), return_counts=True)

(array([0., 1., 2.], dtype=float32), array([1450, 3702, 2590]))